# Importing Libraries

In [1]:
import sys
import os
import json
import numpy as np
from tqdm import tqdm
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import  generateTargetDataBuySide, generateTargetDataSellSide, getTechnicalIndicators, normalize_dataframe_with_mean_std, normalize_new_row_with_mean_std
from utils_data import UpstoxStockDataFetcher

# Load data

In [2]:
fetcher = UpstoxStockDataFetcher()
stock_symbol = "SUZLON"
start_date = "2025-06-15"
end_date = "2025-06-18"
df = fetcher.get_stock_data(stock_symbol, start_date, end_date)
df 


,time,open,high,low,close,volume
0,2025-06-16 09:15:00+05:30,64.50,65.05,64.45,65.05,1860104
1,2025-06-16 09:16:00+05:30,65.07,65.30,64.86,64.97,1193739
2,2025-06-16 09:17:00+05:30,64.96,65.07,64.88,64.92,571128
3,2025-06-16 09:18:00+05:30,64.95,65.01,64.60,64.66,772514
4,2025-06-16 09:19:00+05:30,64.66,64.69,64.38,64.57,1089367
...,...,...,...,...,...,...
1120,2025-06-18 15:25:00+05:30,63.96,63.96,63.91,63.94,118175
1121,2025-06-18 15:26:00+05:30,63.94,63.94,63.89,63.91,262339
1122,2025-06-18 15:27:00+05:30,63.89,63.92,63.86,63.89,264268
1123,2025-06-18 15:28:00+05:30,63.89,63.91,63.88,63.89,254881


In [3]:
df = getTechnicalIndicators(df)
target_buy = generateTargetDataBuySide(df,1.005,0.99)
target_sell = generateTargetDataSellSide(df,0.995,1.01)


Processing Sell Side Data: 100%|██████████| 1098/1098 [00:41<00:00, 26.67it/s]


In [4]:
count1 = len(target_buy[target_buy['action'].isin(['End of Day'])])
count2 = len(target_buy[target_buy['action'].isin(['Target Hit'])])
count3 = len(target_buy[target_buy['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count1}")
print(f"Target Hit: {count2}")
print(f"Stop Loss Hit: {count3}")

End of Day: 313
Target Hit: 604
Stop Loss Hit: 181


In [5]:
count4 = len(target_sell[target_sell['action'].isin(['End of Day'])])
count5 = len(target_sell[target_sell['action'].isin(['Target Hit'])])
count6 = len(target_sell[target_sell['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count4}")
print(f"Target Hit: {count5}")
print(f"Stop Loss Hit: {count6}")

End of Day: 373
Target Hit: 472
Stop Loss Hit: 253


In [6]:
with open('C:/Users/srija/Assignment/Trading/json_files/suzlon_14_june_norm_params.json', 'r') as f:
    norm_param = json.load(f)
df_normalized = df.apply(lambda x:  normalize_new_row_with_mean_std(x,norm_param), axis=1)
df_normalized 

,time,open,high,low,close,volume,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
0,42,63.42,63.54,63.40,63.54,0.148690,-0.311075,-2.278575,-5.195724,0.074711,-1.269002,2.170878,-0.438213,4.368766,-1.354629,-3.515543
1,43,63.52,63.73,63.46,63.73,-0.138830,-0.367216,-1.500105,-4.619538,-0.058979,-1.226689,1.851027,-0.126304,4.388066,0.752306,-3.458713
2,44,63.73,63.74,63.53,63.58,0.479474,-0.426334,-1.829013,-4.322464,-0.193186,-1.191916,1.548778,-0.120591,4.294710,0.319508,-3.574071
3,45,63.56,63.59,63.35,63.45,1.386853,-0.488855,-2.091144,-4.219633,-0.316810,-1.170754,1.352010,-0.452894,4.263660,-0.222588,-3.775320
4,46,63.45,63.45,63.31,63.35,0.771457,-0.544571,-2.280925,-4.228426,-0.425898,-1.163557,1.186530,-0.689508,4.049368,-0.168742,-3.918317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,385,63.96,63.96,63.91,63.94,-0.228643,-0.218782,-1.120708,-0.672131,-0.282006,-0.202914,-0.538767,-1.354854,-0.471364,-0.971581,-0.068013
1094,386,63.94,63.94,63.89,63.91,0.394206,-0.231117,-1.319971,-0.775502,-0.275001,-0.233757,-0.432303,-1.301452,-0.514356,-1.239627,-0.175300
1095,387,63.89,63.92,63.86,63.89,0.402540,-0.243025,-1.449638,-0.873814,-0.266056,-0.265315,-0.297612,-1.238643,-0.535730,-1.079488,-0.283376
1096,388,63.89,63.91,63.88,63.89,0.361984,-0.257911,-1.449638,-0.939220,-0.265275,-0.292475,-0.172541,-1.040919,-0.611217,-1.079488,-0.179139


In [7]:
def get_state(df, current_step):
    row = df.iloc[current_step]
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state

# Loading Model

In [8]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #'cuda' if torch.cuda.is_available() else

policy_net.to(device)
target_net.to(device)

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [52]:
model_path = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june\suzlon_14_june_2866.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

# Evaluate Model

In [53]:
import time
correct = 0
wrong = 0
blunder =0
latency_list = []
action_list = []
blunder_action = []
for i in range(0,len(df_normalized)):
    t1 = time.time()
    state = get_state(df_normalized,i)
    with torch.no_grad():
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        q_values = policy_net(state)
        action=q_values.argmax().item()
    action_list.append(action)
    ideal_action = 0
    if target_buy.iloc[i]['action'] == "Target Hit":
        ideal_action = 1
    elif target_sell.iloc[i]['action'] == "Target Hit":
        ideal_action = 2
    if action==ideal_action:
        correct += 1
    elif action == 1 and ideal_action ==2:
        blunder_action.append(action)
        blunder += 1
    elif action == 2 and ideal_action ==1:
        blunder_action.append(action)
        blunder += 1    
    else:
        wrong += 1
    t2 = time.time()
    latency = t2 - t1
    latency_list.append(latency)

print(f"Correct: {correct}, Wrong: {wrong}, Blunder: {blunder}")  
# print(f"Average Latency: {np.mean(latency_list)} seconds")  

print(f"Action Distribution: {np.bincount(action_list)}")  # Count occurrences
print(f"Blunder Action Distribution: {np.bincount(blunder_action)}")  # Count occurrences

Correct: 397, Wrong: 571, Blunder: 130
Action Distribution: [669 190 239]
Blunder Action Distribution: [  0  26 104]


In [ ]:
# Correct: 683, Wrong: 183, Blunder: 232
# Action Distribution: [189 506 403]
# Blunder Action Distribution: [  0  86 146]